In [1]:
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
import matplotlib.pyplot as plt
from ProPaST import ProPaST
from sklearn.decomposition import PCA
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
os.environ['R_HOME'] ='/home/maqinlin/miniconda3/envs/lib/R'


In [ ]:
file_fold = '/home/maqinlin/project/Data/osmfish/osmfish.h5ad'
n_clusters = 11
adata = sc.read_h5ad(file_fold)
adata.var_names_make_unique()
model = ProPaST.ProPaST(adata, n_clusters, device=device)
adata = model.train()


radius = 50
tool = 'mclust' 
from ProPaST.utils import clustering 
if tool == 'mclust':
    clustering(adata, n_clusters, radius=radius, method=tool, refinement=True) 
elif tool == 'leiden':
    clustering(adata, n_clusters, radius=radius, method=tool, start=0.1, end=10, increment=0.01, refinement=False)

    
adata = adata[~pd.isnull(adata.obs['ground_truth'])].copy()

     
pred = adata.obs['domain']
true = adata.obs['ground_truth']

       
ARI = metrics.adjusted_rand_score(true, pred)
NMI = metrics.normalized_mutual_info_score(true, pred)
AMI = metrics.adjusted_mutual_info_score(true, pred)
HOMO = metrics.homogeneity_score(true, pred)
COMP = metrics.completeness_score(true, pred)
FMI = metrics.fowlkes_mallows_score(true, pred)

print(f'ARI: {ARI:.4f}, NMI: {NMI:.4f}, AMI: {AMI:.4f}, Homogeneity: {HOMO:.4f}, Completeness: {COMP:.4f}, FMI: {FMI:.4f}')